In [379]:
from collections import Counter
import random
import os
import subprocess
import sys
import csv

In [380]:

class fasta:
    
    GC_content = 0
    c = {}
    transcription = ''
    translation = ''
    translation_nospaces = ''
    is_dna = True
    molecular_weight = float(0)
    coronavirus = False
    
    def __init__(self, name, sequence):
        self.name = name
        self.sequence = sequence
        self.c = Counter(sequence)
        self.checkDNA_orProtein()
        self.checkCoronavirus()
        
        if self.is_dna == True:
            self.transcription = self.transcribe()
            self.translation = self.translate()
        else:
            self.translation = self.sequence
            self.translation_nospaces = self.sequence
        
    
    def checkCoronavirus(self):
        if 'coronavirus' in self.name:
            self.coronavirus = True
        else:
            self.coronavirus = False
        
        
    def print_translation(self, sequence):
        if len(sequence) > 50:
            return sequence[0:50] + "...."
        else:
            return sequence
    
    def getLength(sequence):
        return len(sequence)
    
    def checkDNA_orProtein(self):
        DNA_set = {'A', 'C', 'G', 'T'}
        sequence_set = set(list(sequence))
        difference = sequence_set - DNA_set
        
        if len(difference) != 0:
            self.is_dna = False
        
    def getAccession(self):
        first_index = self.name.index('|')
        return self.name[0:first_index]
        
        
    def toString(self):
        return self.name + " " + self.sequence
    
    def get_GC_content(self):
        G_content = self.c.get('G')
        C_content = self.c.get('C')
        GC_content = (G_content + C_content)/len(self.sequence)
        return "{:.6%}".format(GC_content)
        
    def transcribe(self):
        dna_string = self.sequence
        dna_string = dna_string.replace('T', 'U')
        return dna_string
    
    def printDNACounter(self):
        return 'A = ' + str(self.c.get('A')), 'C = ' + str(self.c.get('C')), 'G = ' + str(self.c.get('G')), 'T = ' + str(self.c.get('T'))
    
    def getName(self):
        first_index = self.name.index('[')
        second_index = self.name.index(']')
        return self.name[first_index+1:second_index]
        
    
    def translate(self):
        rna = self.transcription
        codon_table = {"UUU":"F", "UUC":"F", "UUA":"L", "UUG":"L",
            "UCU":"S", "UCC":"S", "UCA":"S", "UCG":"S",
            "UAU":"Y", "UAC":"Y", "UAA":"Stop", "UAG":"Stop",
            "UGU":"C", "UGC":"C", "UGA":"Stop", "UGG":"W",
            "CUU":"L", "CUC":"L", "CUA":"L", "CUG":"L",
            "CCU":"P", "CCC":"P", "CCA":"P", "CCG":"P",
            "CAU":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
            "CGU":"R", "CGC":"R", "CGA":"R", "CGG":"R",
            "AUU":"I", "AUC":"I", "AUA":"I", "AUG":"M",
            "ACU":"T", "ACC":"T", "ACA":"T", "ACG":"T",
            "AAU":"N", "AAC":"N", "AAA":"K", "AAG":"K",
            "AGU":"S", "AGC":"S", "AGA":"R", "AGG":"R",
            "GUU":"V", "GUC":"V", "GUA":"V", "GUG":"V",
            "GCU":"A", "GCC":"A", "GCA":"A", "GCG":"A",
            "GAU":"D", "GAC":"D", "GAA":"E", "GAG":"E",
            "GGU":"G", "GGC":"G", "GGA":"G", "GGG":"G",}
            
        amino_acid_chain = []
        for i in range(0,len(rna),3):
            codon = rna[i:i+3]
            amino_acid = codon_table.get(codon)
            amino_acid_chain.append(amino_acid)
        
        amino_acid_chain_string = '-'.join(str(x) for x in amino_acid_chain)
        self.translation_nospaces = amino_acid_chain_string.split('-')
        return amino_acid_chain_string
    
    def calculateMolecularWeight(self):
        aa = {'A' : 71.03711,'R' : 156.10111,'N' : 114.04293,'D' : 115.02694,'C' : 103.00919,'E' : 129.04259,
              'Q' : 128.05858,'G' : 57.02146,'H' : 137.05891,'I' : 113.08406,'L' : 113.08406,'K' : 128.09496,
              'M' : 131.04049,'F' : 147.06841,'P' : 97.05276,'S' : 87.03203,'T' : 101.04768,'W' : 186.07931,
              'Y' : 163.06333,'V' : 99.06841, 'X' : 0.00000  }
        protein_sequence = self.translation_nospaces
        molecular_weight = float(0)
        
        for amino_acid in protein_sequence:
            aa_weight = aa.get(amino_acid) 
            if aa_weight != None:
                molecular_weight += aa_weight
            else:
                continue
            
        self.molecular_weight = molecular_weight
        return molecular_weight
        
    
    def printDNA_analysis(self):
        print('\n Name:  ', self.getName())
        print('\n \t DNA Analysis: ')
        print('--------------------------')
        print('\t DNA sequence: ', self.sequence)
        print('\t', 'Transcription: ',self.transcription)
        print('\t', 'Number of each base in DNA: ', self.printDNACounter())
        print('\t GC content: ', self.get_GC_content())
    
            
    def printProtein_analysis(self):
        print('\n \t Protein Analysis: ')
        print('--------------------------')
        print('\n \t Amino Acid Chain: ', self.print_translation(self.translation))
        print('\t Number of Amino acids: ', len(self.translation_nospaces))
        print('\t Molecular Weight: ', self.calculateMolecularWeight(), 'Da')
        
    

In [381]:

f = open('basic_ds.fasta', 'r')
f2 = open('related_ds.fasta', 'r')

def file_into_fasta_records(f):
    lines = f.readlines()
    oneString = ''
    for word in lines:
        oneString = oneString + word

    records = oneString.split('>')

    fasta_records = []
    for x in range(1,len(records)):
        record = records[x]
        record_name_index = records[x].find('\n')
        name = record[0:record_name_index]
        name = name.replace('\n', '')
        sequence = record[record_name_index:]
        sequence = sequence.replace('\n', '')
        class_record = fasta(name, sequence)
        fasta_records.append(class_record)
        
    print('Number of records: ', len(fasta_records))
    return fasta_records

def fasta_records_analysis(fasta_records):
    for x in fasta_records:
        if x.is_dna == True:
            x.printDNA_analysis()
            x.printProtein_analysis()
        else:
            print('\n Name:  ', x.getName())
            x.printProtein_analysis()

In [382]:
fasta_records_basic_ds = file_into_fasta_records(f)
fasta_records_analysis(fasta_records_basic_ds)

fasta_records_related_ds = file_into_fasta_records(f2)
fasta_records_analysis(fasta_records_related_ds)
                
        

Number of records:  25

 Name:   Acholeplasma virus L2

 	 Protein Analysis: 
--------------------------

 	 Amino Acid Chain:  DTIHGPDIVHKQTSSIVTINDIILLYESDGRFVSVAEDGYSGNGARVGTY....
	 Number of Amino acids:  212
	 Molecular Weight:  23228.095139999983 Da

 Name:   Swine enteric coronavirus

 	 Protein Analysis: 
--------------------------

 	 Amino Acid Chain:  MTFPRALTVIDDNGVVISIIFWFLLIIILILLSIALLNIIKLCMVCCNLG....
	 Number of Amino acids:  82
	 Molecular Weight:  9266.163529999996 Da

 Name:   Human papillomavirus type 10

 	 Protein Analysis: 
--------------------------

 	 Amino Acid Chain:  MSMGAQEPRNILLLCRNCGIPLEDLRLCCIFCTKQLTAAELAAFALRELY....
	 Number of Amino acids:  148
	 Molecular Weight:  17533.822229999994 Da

 Name:   Antheraea pernyi nucleopolyhedrovirus

 	 Protein Analysis: 
--------------------------

 	 Amino Acid Chain:  MFELIICAALVGAGFADPLSFEPIDDASGLVFERLAALRHVSDERFVFIK....
	 Number of Amino acids:  654
	 Molecular Weight:  73367.18372000022 Da

 Name:   Avian leuke

In [383]:
def write_fasta(fasta_records, length, file, length_of_first):
    if length_of_first == True:
        for i in range(0,length):
            file.write('\n\n>' + fasta_records[i].name)
            file.write('\n' + fasta_records[i].sequence)
    else:
        for i in range(length, len(fasta_records)):
            file.write('\n\n>' + fasta_records[i].name)
            file.write('\n' + fasta_records[i].sequence)
        

In [384]:
eighty_percent_basic_ds = open('eighty_percent_basic_ds.fasta', 'w')
twenty_percent_basic_ds = open('twenty_percent_basic_ds.fasta', 'w')

try:
    os.mkdir("split_datasets_basic")
except FileExistsError: 
    print("File already exists, will continue")
    
os.rename('eighty_percent_basic_ds.fasta', "split_datasets_basic/eighty_percent_basic_ds.fasta")
os.rename('twenty_percent_basic_ds.fasta', "split_datasets_basic/twenty_percent_basic_ds.fasta")

eighty_percent_length = int(len(fasta_records_basic_ds) * .8)
twenty_percent_length = len(fasta_records_basic_ds) - eighty_percent_length


write_fasta(fasta_records_basic_ds, eighty_percent_length, eighty_percent_basic_ds, True)
write_fasta(fasta_records_basic_ds, len(fasta_records_basic_ds) - twenty_percent_length, twenty_percent_basic_ds, False)


eighty_percent_basic_ds.close()
twenty_percent_basic_ds.close()

eighty_percent_related_ds = open('eighty_percent_related_ds.fasta', 'w')
twenty_percent_related_ds = open('twenty_percent_related_ds.fasta', 'w')

try:
    os.mkdir("split_datasets_related")
except FileExistsError: 
    print("File already exists, will continue")
    
os.rename('eighty_percent_related_ds.fasta', "split_datasets_related/eighty_percent_basic_ds.fasta")
os.rename('twenty_percent_related_ds.fasta', "split_datasets_related/twenty_percent_basic_ds.fasta")

eighty_percent_length = int(len(fasta_records_related_ds) * .8)
twenty_percent_length = len(fasta_records_related_ds) - eighty_percent_length


write_fasta(fasta_records_related_ds, eighty_percent_length, eighty_percent_related_ds, True)
write_fasta(fasta_records_related_ds, len(fasta_records_related_ds) - twenty_percent_length, twenty_percent_related_ds, False)


eighty_percent_related_ds.close()
twenty_percent_related_ds.close()




File already exists, will continue
File already exists, will continue


In [385]:
fasta_records_csv  = []
fields = ['Accession_id','Species', 'Length', 'Molecular weight(Da)', 'Coronavirus']
for x in fasta_records_basic_ds:
    record = [x.getAccession(), x.getName(), len(x.translation_nospaces), x.calculateMolecularWeight(), int(x.coronavirus)]
    fasta_records_csv.append(record)

file_name_csv = 'virus.csv'
with open(file_name_csv, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(fasta_records_csv)